#Project Outline

- scrape www.github.com/topics
- get a list of topics
- get topic title, url, description
- get top 25 repo for each topic
- for each repo, get repo name, username, stars, repo url
- create csv file for each topic with columns as:

repo_name -- username -- repo -- url

#Use Request library to download webpages

In [926]:
import requests

In [927]:
url = "https://github.com/topics"

In [928]:
response = requests.get(url)

In [929]:
response.status_code

200

In [930]:
content = response.text
content

'\n\n<!DOCTYPE html>\n<html lang="en" data-color-mode="auto" data-light-theme="light" data-dark-theme="dark"  data-a11y-animated-images="system">\n  <head>\n    <meta charset="utf-8">\n  <link rel="dns-prefetch" href="https://github.githubassets.com">\n  <link rel="dns-prefetch" href="https://avatars.githubusercontent.com">\n  <link rel="dns-prefetch" href="https://github-cloud.s3.amazonaws.com">\n  <link rel="dns-prefetch" href="https://user-images.githubusercontent.com/">\n  <link rel="preconnect" href="https://github.githubassets.com" crossorigin>\n  <link rel="preconnect" href="https://avatars.githubusercontent.com">\n\n  \n\n  <link crossorigin="anonymous" media="all" rel="stylesheet" href="https://github.githubassets.com/assets/light-8cafbcbd78f4.css" /><link crossorigin="anonymous" media="all" rel="stylesheet" href="https://github.githubassets.com/assets/dark-31dc14e38457.css" /><link data-color-theme="dark_dimmed" crossorigin="anonymous" media="all" rel="stylesheet" data-href="

In [931]:
with open('webpage.html','w') as f:
  f.write(content)

#Use BeautifulSoup to parse & extract information from html page

In [932]:
from bs4 import BeautifulSoup

In [933]:
soup1 = BeautifulSoup(content,"html.parser")

In [934]:
# import pandas as pd

# box = soup1.find_all('div', class_='py-4 border-bottom d-flex flex-justify-between')

# data = []

# for i in box:
#   topics = i.find('p', class_='f3 lh-condensed mb-0 mt-1 Link--primary').text
#   desc = i.find('p', class_='f5 color-fg-muted mb-0 mt-1').text.replace('\n', '')
#   link = "https://github.com/" + i.find('a', class_='no-underline flex-grow-0')['href']
#   data.append({'Topics': topics, 'Description': desc, 'Link': link})


# df = pd.DataFrame()
# for item in data:
#   df = df.append(item, ignore_index=True)






  #Getting information from each topic link


    # response = requests.get(title_link)
    # content = response.text
    # soup = BeautifulSoup(content, "html.parser")
    # first = soup.find_all('article', class_ = 'border rounded color-shadow-small color-bg-subtle my-4')

    # topic_data = []

  # for k in first:
  #   owner = k.find('a', {'data-hydro-click': True}).text.strip()
  #   repo = k.find('a', {'class': 'text-bold wb-break-word'}).text.strip()
  #   topic_link = f"https://github.com/{owner}/{repo}"
    # star_box = soup.find_all('div', class_ = 'd-flex flex-items-center')
    # for l in star_box:
    #   star = l.find('span',class_="Counter js-social-count").text.replace('k','')
    #   star = (float(star)*1000)
    #   topic_data.append({'topic':topics, 'owner':owner, 'repo':repo, 'star':star})

In [935]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

box = soup1.find_all('div', class_='py-4 border-bottom d-flex flex-justify-between')

data = []

for i in box:
    topics = i.find('p', class_='f3 lh-condensed mb-0 mt-1 Link--primary').text
    desc = i.find('p', class_='f5 color-fg-muted mb-0 mt-1').text.replace('\n', '')
    link = "https://github.com/" + i.find('a', class_='no-underline flex-grow-0')['href']
    data.append({'Topics': topics, 'Description': desc, 'Link': link})

df = pd.DataFrame(data)

# Function to scrape data for each link
def scrape_topic_data(row):
    response = requests.get(row['Link'])
    content = response.text
    soup = BeautifulSoup(content, "html.parser")
    first = soup.find_all('article', class_='border rounded color-shadow-small color-bg-subtle my-4')

    topic_data = []

    for k in first:
        owner = k.find('a', {'data-hydro-click': True}).text.strip()
        repo = k.find('a', {'class': 'text-bold wb-break-word'}).text.strip()
        topic_link = f"https://github.com/{owner}/{repo}"
        star_box = k.find_all('div', class_='d-flex flex-items-center')

        for l in star_box:
            star = l.find('span', class_="Counter js-social-count").text
            topic_data.append({'topic': row['Topics'],'desc': row['Description'],'link': row['Link'], 'owner': owner, 'repo': repo, 'star': star, 'topic-link':topic_link})

    return pd.DataFrame(topic_data)

# Create a dictionary of DataFrames, where each key is a topic
dfs = {}
for _, row in df.iterrows():
    topic_df = scrape_topic_data(row)

    topic = row['Topics']
    topic_df.sort_values('star', ascending=True, inplace=True)
    topic_df = topic_df.head(3)
    dfs[topic] = topic_df

# Access each DataFrame by topic
# for topic, topic_df in dfs.items():
#     print(f"Topic: {topic}")
#     print(topic_df)
#     print()



# Create an Excel writer object
writer = pd.ExcelWriter('topics_data.xlsx', engine='xlsxwriter')

# Write each DataFrame to a separate sheet in the workbook
for topic, topic_df in dfs.items():
    topic_df.to_excel(writer, sheet_name=topic, index=False)

# Save the workbook
writer.save()

<ipython-input-935-483528f23fd1>:64: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
